In [4]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

In [5]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [6]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

gripper = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
cap = parser.AddModels(file_name="my_sdfs/bottle_cap.sdf")[0]

plant.Finalize()

In [7]:
B_cap = plant.GetBodyByName("cap_link", cap)
B_gripper = plant.GetBodyByName("body", gripper)


In [8]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

In [9]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [10]:
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

p_GgraspO = [0, .115, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi / 2).multiply(RotationMatrix.MakeZRotation(np.pi / 2))

X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

plant.SetFreeBodyPose(plant_context, B_gripper, X_WGgrasp)

plant.GetJointByName("left_finger_sliding_joint", gripper).set_translation(
    plant_context, -0.033
)
plant.GetJointByName("right_finger_sliding_joint", gripper).set_translation(
    plant_context, 0.033
)

<PrismaticJoint name='right_finger_sliding_joint' index=1 model_instance=2>

In [11]:
diagram.ForcedPublish(context)